In [35]:
import pandas as pd

ohlcv_data = pd.read_csv("/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_raw_data.csv", parse_dates=['time'])
ohlcv_data = ohlcv_data.set_index('time')
ohlcv_data['year'] = ohlcv_data.index.year
ohlcv_data['month'] = ohlcv_data.index.month
ohlcv_data['day_of_week'] = ohlcv_data.index.dayofweek
ohlcv_data['hour'] = ohlcv_data.index.hour
ohlcv_data['minute'] = ohlcv_data.index.minute
start_date = '2007-01-01'
end_date = '2015-01-01'
ohlcv_data = ohlcv_data[start_date:end_date]
df = ohlcv_data.copy()
df

,open,high,low,close,volume,year,month,day_of_week,hour,minute
time,,,,,,,,,,
2007-01-01 21:30:00+00:00,1.95840,1.95860,1.95840,1.95840,18,2007,1,0,21,30
2007-01-01 21:45:00+00:00,1.95840,1.95965,1.95830,1.95920,47,2007,1,0,21,45
2007-01-01 22:00:00+00:00,1.95910,1.96005,1.95890,1.95978,60,2007,1,0,22,0
2007-01-01 22:15:00+00:00,1.96003,1.96190,1.96003,1.96156,91,2007,1,0,22,15
2007-01-01 22:30:00+00:00,1.96138,1.96174,1.96083,1.96174,82,2007,1,0,22,30
...,...,...,...,...,...,...,...,...,...,...
2015-01-01 22:45:00+00:00,1.55835,1.55842,1.55732,1.55794,118,2015,1,3,22,45
2015-01-01 23:00:00+00:00,1.55800,1.55841,1.55722,1.55828,214,2015,1,3,23,0
2015-01-01 23:15:00+00:00,1.55832,1.55832,1.55694,1.55704,589,2015,1,3,23,15


In [ ]:
# Convert the 'Date' column to datetime
ohlcv_data['Date'] = pd.to_datetime(ohlcv_data['Date'])

# Set the 'Date' column as the index
ohlcv_data.set_index('Date', inplace=True)

# Define the start date and the duration for the data extraction
start_date = '2007-01-01'
duration_months = 6

# Calculate the end date by adding the duration to the start date
end_date = pd.to_datetime(start_date) + pd.DateOffset(months=duration_months)

# Extract the data between the start date and the end date
six_months_data = ohlcv_data.loc[start_date:end_date]

# Display the extracted data
print(six_months_data)

In [42]:
df

,open,high,low,close,volume,year,month,day_of_week,hour,minute
time,,,,,,,,,,
2007-01-01 21:30:00+00:00,1.95840,1.95860,1.95840,1.95840,18,2007,1,0,21,30
2007-01-01 21:45:00+00:00,1.95840,1.95965,1.95830,1.95920,47,2007,1,0,21,45
2007-01-01 22:00:00+00:00,1.95910,1.96005,1.95890,1.95978,60,2007,1,0,22,0
2007-01-01 22:15:00+00:00,1.96003,1.96190,1.96003,1.96156,91,2007,1,0,22,15
2007-01-01 22:30:00+00:00,1.96138,1.96174,1.96083,1.96174,82,2007,1,0,22,30
...,...,...,...,...,...,...,...,...,...,...
2015-01-01 22:45:00+00:00,1.55835,1.55842,1.55732,1.55794,118,2015,1,3,22,45
2015-01-01 23:00:00+00:00,1.55800,1.55841,1.55722,1.55828,214,2015,1,3,23,0
2015-01-01 23:15:00+00:00,1.55832,1.55832,1.55694,1.55704,589,2015,1,3,23,15


In [54]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

ONE_DAY = 4*24   
train_size = 80 * ONE_DAY # 80 days 
test_size = 10 * ONE_DAY # 10 days
val_test_horizon = test_size * 2
splitter = SlidingWindowSplitter(window_length=train_size, 
                                 fh=np.arange(1, val_test_horizon + 1),
                                 step_length=test_size)

for i, (train_idx, val_test_idx) in enumerate(splitter.split(df)):
    val_idx = val_test_idx[:test_size]
    test_idx = val_test_idx[test_size:]
    
    df_train = df.iloc[train_idx, :].to_numpy()
    df_val = df.iloc[val_idx, :].to_numpy()
    df_test = df.iloc[test_idx, :].to_numpy()
    
    print(f"Window: {i}")
    print(f"Train: {df_train.shape[0]}")
    print(f"Val: {df_val.shape[0]}")
    print(f"Test: {df_test.shape[0]}")
    print()

Window: 0
Train: 7680
Val: 960
Test: 960

Window: 1
Train: 7680
Val: 960
Test: 960

Window: 2
Train: 7680
Val: 960
Test: 960

Window: 3
Train: 7680
Val: 960
Test: 960

Window: 4
Train: 7680
Val: 960
Test: 960

Window: 5
Train: 7680
Val: 960
Test: 960

Window: 6
Train: 7680
Val: 960
Test: 960

Window: 7
Train: 7680
Val: 960
Test: 960

Window: 8
Train: 7680
Val: 960
Test: 960

Window: 9
Train: 7680
Val: 960
Test: 960

Window: 10
Train: 7680
Val: 960
Test: 960

Window: 11
Train: 7680
Val: 960
Test: 960

Window: 12
Train: 7680
Val: 960
Test: 960

Window: 13
Train: 7680
Val: 960
Test: 960

Window: 14
Train: 7680
Val: 960
Test: 960

Window: 15
Train: 7680
Val: 960
Test: 960

Window: 16
Train: 7680
Val: 960
Test: 960

Window: 17
Train: 7680
Val: 960
Test: 960

Window: 18
Train: 7680
Val: 960
Test: 960

Window: 19
Train: 7680
Val: 960
Test: 960

Window: 20
Train: 7680
Val: 960
Test: 960

Window: 21
Train: 7680
Val: 960
Test: 960

Window: 22
Train: 7680
Val: 960
Test: 960

Window: 23
Train: 768

In [ ]:
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
ts = np.arange(10)
splitter = SlidingWindowSplitter(fh=[2, 4], window_length=3, step_length=2)

for train, test in splitter.split(y=ts):
    print("train, test")
    print(train, test) 

In [ ]:
import numpy as np

class SlidingWindowSplitter:
    def __init__(self, n_samples, train_size, val_size, test_size):
        self.n_samples = n_samples
        self.train_size = train_size
        self.val_size = val_size
        self.test_size = test_size
        self.current_index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current_index + self.train_size + self.val_size + self.test_size > self.n_samples:
            raise StopIteration
        
        train_indices = np.arange(self.current_index, self.current_index + self.train_size)
        val_indices = np.arange(self.current_index + self.train_size, self.current_index + self.train_size + self.val_size)
        test_indices = np.arange(self.current_index + self.train_size + self.val_size, self.current_index + self.train_size + self.val_size + self.test_size)
        
        self.current_index += 1
        
        return train_indices, val_indices, test_indices

# Example usage:
n_samples = 100
train_size = 60
val_size = 20
test_size = 20

splitter = SlidingWindowSplitter(n_samples, train_size, val_size, test_size)

for train_indices, val_indices, test_indices in splitter:
    print("Train indices:", train_indices)
    print("Validation indices:", val_indices)
    print("Test indices:", test_indices)
    print()
